In [1]:
import pymongo
import json
from transformers import pipeline
import time
from collections import defaultdict
import torch
import os
import math
from pymongo import MongoClient
from pymongo.errors import PyMongoError

In [2]:
# ==============================================
# CONFIGURATION
# ==============================================
MONGO_HOST = "localhost"
MONGO_PORT = 27017
MONGO_DB = "PFE"
MONGO_COLLECTION = "resumes"

SKILL_BONUS_THRESHOLD = 2
MAX_BONUS = 0.1

In [3]:
# ==============================================
# FONCTIONS D'INITIALISATION
# ==============================================
def initialize_mongodb_connection():
    """Initialise la connexion à MongoDB"""
    try:
        client = MongoClient(host=MONGO_HOST, port=MONGO_PORT)
        db = client[MONGO_DB]
        collection = db[MONGO_COLLECTION]
        print(f"Connecté à MongoDB: {MONGO_HOST}:{MONGO_PORT}/{MONGO_DB}.{MONGO_COLLECTION}")
        return collection
    except PyMongoError as e:
        print(f"Erreur de connexion à MongoDB: {e}")
        raise

def initialize_classifier():
    """Initialise le classificateur Zero-Shot avec gestion des erreurs"""
    try:
        device = 0 if torch.cuda.is_available() else -1
        print(f"Device set to use cuda:{device}" if device == 0 else "Using CPU")

        return pipeline(
            "zero-shot-classification",
            model="facebook/bart-large-mnli",
            device=device
        )
    except Exception as e:
        print(f"Erreur avec bart-large-mnli: {e}. Tentative avec un modèle alternatif...")
        try:
            return pipeline(
                "zero-shot-classification",
                model="typeform/distilbert-base-uncased-mnli",
                device=0 if torch.cuda.is_available() else -1,
                framework="tf",
                from_tf=True
            )
        except Exception as alt_e:
            print(f"Erreur avec le modèle alternatif: {alt_e}")
            raise RuntimeError("Impossible de charger un modèle de classification")


In [4]:
# ==============================================
# FONCTIONS DE CALCUL
# ==============================================
def calculate_category_bonus(num_skills_in_category):
    """Calcule le bonus pour une catégorie"""
    if num_skills_in_category < SKILL_BONUS_THRESHOLD:
        return 0.0

    ratio = num_skills_in_category / SKILL_BONUS_THRESHOLD
    bonus = MAX_BONUS * (1 - 1/(1 + math.log(ratio + 1)))
    return bonus


In [5]:
# ==============================================
# FONCTIONS DE GESTION DES DONNÉES
# ==============================================
def load_data_from_mongodb(collection):
    """Charge les données depuis MongoDB"""
    try:
        data = list(collection.find({}))
        if not data:
            print("Avertissement: Aucun document trouvé dans la collection MongoDB")
            return []
        return data
    except PyMongoError as e:
        print(f"Erreur lors de la lecture depuis MongoDB: {e}")
        return None

def save_data_to_mongodb(collection, data):
    """Sauvegarde les données dans MongoDB"""
    if not data:
        print("Avertissement: Aucune donnée à sauvegarder")
        return False

    try:
        bulk_operations = []
        for doc in data:
            if '_id' in doc:
                bulk_operations.append(
                    pymongo.ReplaceOne({'_id': doc['_id']}, doc)
                )
            else:
                bulk_operations.append(
                    pymongo.InsertOne(doc)
                )

        if bulk_operations:
            result = collection.bulk_write(bulk_operations)
            print(f"{result.modified_count} documents mis à jour, {result.inserted_count} documents insérés")
            return True
    except PyMongoError as e:
        print(f"Erreur lors de l'écriture dans MongoDB: {e}")
        return False

def normalize_cv_structure(cv):
    """Normalise la structure d'un CV"""
    if not isinstance(cv, dict):
        cv = {"raw_data": cv}

    cv.setdefault("skills", [])
    cv.setdefault("categories", [])
    cv.setdefault("name", "CV sans nom")

    # Normalisation des compétences
    if isinstance(cv["skills"], list):
        normalized_skills = []
        for skill in cv["skills"]:
            if isinstance(skill, dict):
                normalized_skills.append({
                    "name": str(skill.get("name", "")),
                    "score": float(skill.get("score", 0.0))
                })
            elif isinstance(skill, str):
                normalized_skills.append({
                    "name": skill,
                    "score": 0.0
                })
        cv["skills"] = normalized_skills

    # Normalisation des catégories
    if isinstance(cv["categories"], list):
        normalized_categories = []
        for cat in cv["categories"]:
            if isinstance(cat, dict):
                normalized_categories.append({
                    "name": str(cat.get("name", "")),
                    "score": float(cat.get("score", 0.0))
                })
            elif isinstance(cat, str):
                normalized_categories.append({
                    "name": cat,
                    "score": 0.0
                })
        cv["categories"] = normalized_categories

    return cv


In [6]:
# ==============================================
# FONCTIONS DE TRAITEMENT
# ==============================================
def process_single_cv(cv, classifier):
    """Traite un seul CV en appliquant le bonus par catégorie"""
    cv = normalize_cv_structure(cv)

    if not cv["skills"] or not cv["categories"]:
        print(f"Avertissement: CV '{cv['name']}' n'a pas de compétences ou de catégories")
        return cv

    try:
        skills = [skill["name"] for skill in cv["skills"] if skill["name"]]
        skill_scores = {skill["name"]: skill["score"] for skill in cv["skills"]}
        existing_categories = [cat["name"] for cat in cv["categories"] if cat["name"]]

        if not skills or not existing_categories:
            return cv
    except Exception as e:
        print(f"Erreur lors de la préparation des données pour {cv['name']}: {e}")
        return cv

    category_data = defaultdict(lambda: {"scores": [], "skills": []})

    for skill in skills:
        try:
            result = classifier(
                skill,
                existing_categories,
                multi_label=True,
                truncation=True
            )
            for category, score in zip(result["labels"], result["scores"]):
                if score > 0.5:
                    category_data[category]["scores"].append(skill_scores.get(skill, 0))
                    category_data[category]["skills"].append(skill)
        except Exception as e:
            print(f"Erreur lors de la classification de {skill}: {e}")
            continue

    updated_categories = []
    for cat in cv["categories"]:
        cat_name = cat["name"]
        if cat_name in category_data and category_data[cat_name]["scores"]:
            num_skills_in_category = len(category_data[cat_name]["skills"])
            avg_score = sum(category_data[cat_name]["scores"]) / num_skills_in_category
            bonus = calculate_category_bonus(num_skills_in_category)

            updated_score = avg_score * (1 + bonus)
            updated_score = min(1.0, updated_score)
            updated_categories.append({
                "name": cat_name,
                "score": updated_score
            })
            print(f"Catégorie '{cat_name}' a {num_skills_in_category} compétences → bonus: {bonus}")
        else:
            updated_categories.append({
                "name": cat_name,
                "score": round(cat["score"], 7),
            })

    cv["categories"] = updated_categories
    cv["last_updated"] = time.strftime("%Y-%m-%dT%H:%M:%SZ", time.gmtime())

    return cv

def process_all_cvs(data, classifier, batch_size=5, delay=2):
    """Traite tous les CVs dans les données de MongoDB"""
    if not isinstance(data, list):
        if isinstance(data, dict):
            data = [data]
        else:
            print("Erreur: Les données doivent être une liste ou un dictionnaire")
            return None

    total_cvs = len(data)
    print(f"Début du traitement de {total_cvs} CVs...")

    processed = 0
    updated_cvs = []

    for cv in data:
        try:
            start_time = time.time()
            processed_cv = process_single_cv(cv, classifier)
            updated_cvs.append(processed_cv)

            processed += 1
            elapsed = time.time() - start_time
            print(f"CV traité {processed}/{total_cvs} ({elapsed:.2f}s): {processed_cv['name']}")

            if processed % batch_size == 0:
                print(f"Pause de {delay} seconde(s)...")
                time.sleep(delay)

        except Exception as e:
            print(f"Erreur lors du traitement du CV: {e}")
            updated_cvs.append(normalize_cv_structure(cv))
            continue

    print("Traitement terminé!")
    return updated_cvs

In [7]:
# ==============================================
# FONCTION PRINCIPALE
# ==============================================
def main():
    """Fonction principale"""
    try:
        collection = initialize_mongodb_connection()
    except Exception as e:
        print(f"Échec de la connexion à MongoDB: {e}")
        return

    try:
        classifier = initialize_classifier()
        print("Modèle de classification chargé avec succès")
    except Exception as e:
        print(f"Échec du chargement du modèle: {e}")
        return

    input_data = load_data_from_mongodb(collection)
    if input_data is None:
        print("Impossible de charger les données depuis MongoDB")
        return

    if isinstance(input_data, list):
        output_data = process_all_cvs(input_data, classifier)
    else:
        output_data = process_single_cv(input_data, classifier)
        if output_data is not None:
            output_data = [output_data]

    if output_data is not None:
        if save_data_to_mongodb(collection, output_data):
            print("Résultats sauvegardés dans MongoDB avec succès")
        else:
            print("Échec de la sauvegarde des résultats dans MongoDB")

if __name__ == "__main__":
    main()

Connecté à MongoDB: localhost:27017/PFE.resumes
Using CPU


Device set to use cpu


Modèle de classification chargé avec succès
Début du traitement de 44 CVs...
CV traité 1/44 (16.81s): Alice Clark
Catégorie 'Cloud Practitioner' a 12 compétences → bonus: 0.06605463339332744
Catégorie 'JavaScript Developer' a 6 compétences → bonus: 0.05809402158035948
Catégorie 'Python Developer' a 3 compétences → bonus: 0.047815851563296555
Catégorie 'SQL Expert' a 1 compétences → bonus: 0.0
CV traité 2/44 (46.27s): Chia Yong Kang
CV traité 3/44 (1.58s): Michael Smith
Catégorie 'UNKNOWN' a 1 compétences → bonus: 0.0
CV traité 4/44 (12.09s): xinni chng
Catégorie 'Cloud Practitioner' a 3 compétences → bonus: 0.047815851563296555
Catégorie 'JavaScript Developer' a 3 compétences → bonus: 0.047815851563296555
Catégorie 'Python Developer' a 1 compétences → bonus: 0.0
CV traité 5/44 (25.63s): Ashly Lau
Pause de 2 seconde(s)...
Catégorie 'JavaScript Developer' a 1 compétences → bonus: 0.0
Catégorie 'SQL Expert' a 1 compétences → bonus: 0.0
CV traité 6/44 (10.97s): Thomas Frank
Catégorie 'Clou